In [113]:
import numpy as np
from numba import jit, prange
import timeit

In [114]:
def kron_gates_l(single_gates):
    result = single_gates[0]


    for gate in single_gates[1:]:

        result = np.kron(result, gate)

    return result


@jit(parallel=True)
def kron_neighbours_even(single_gates):

    l, dims, _ = single_gates.shape

    double_gates = np.zeros((l // 2, dims**2, dims**2), dtype=np.complex128)

    for i in prange(0, l // 2):
        double_gates[i, :, :] = np.kron(single_gates[i * 2], single_gates[i * 2 + 1])

    return double_gates


def kron_gates_t(single_gates):
    """Recursively multiply the neighbouring gates.
    When the block size gets below the turnover point the linear
    kron_gates_l is used as it is more efficient in this usecase."""
    TURNOVER = 3

    l = len(single_gates)

    if l > TURNOVER:
        if l % 2 == 0:
            return kron_gates_t(kron_neighbours_even(single_gates))
        return np.kron(
            kron_gates_t(kron_neighbours_even(single_gates[:-1, :, :])),
            single_gates[-1],
        )

    return kron_gates_l(np.array(single_gates))

In [115]:
def rz(thetas):

    zero = np.zeros(thetas.shape)
    exp_m_theta = np.exp(-1j * thetas / 2)
    exp_theta = np.exp(1j * thetas / 2)

    single_gates = np.einsum(
        "ijk->kji", np.array([[exp_m_theta, zero], [zero, exp_theta]]), optimize="greedy",
    order="C"
    )

    u_gates = kron_gates_t(single_gates)

    return u_gates

In [116]:
from pykronecker import KroneckerProduct
def rz3(thetas):

    zero = np.zeros(thetas.shape)
    exp_m_theta = np.exp(-1j * thetas / 2)
    exp_theta = np.exp(1j * thetas / 2)

    single_gates = np.einsum(
        "ijk->kji",
        np.array([[exp_m_theta, zero], [zero, exp_theta]]),
        optimize="greedy",
        # order="C",
    )

    u_gates = KroneckerProduct(single_gates)

    return u_gates

In [117]:
def rz2(thetas):

    gate = np.array([1])

    for theta in thetas:
        single_gate = np.array([[np.exp(-1j * theta / 2), 0], [0, np.exp(1j * theta / 2)]])
        gate = np.kron(gate, single_gate)

    return gate

In [124]:
%timeit  rz3(np.ones(7)) @ rz3(np.ones(7)+1) @ rz3(np.ones(7)+2 ).to_array()

4.32 ms ± 99.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [125]:
%timeit rz(np.ones(7)) @ rz(np.ones(7)+1) @ rz(np.ones(7)+2 )

1.51 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit rz2(np.ones(7))

346 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
